# スポーツジムの顧客データを分析３
ここでは既に退会してしまった顧客と継続して利用している顧客のデータを用いて、顧客の退会を予測します。  
決定木分析を用いて予測モデルを作成し、１ヶ月前の顧客データから翌月の退会予測を行います。


### このジムは、月末までに退会申請を提出することで、翌月末に退会することが出来ます。
- 退会前月(退会申請) &emsp;&nbsp;:&nbsp;2018年08月
- 退会月(退会申請済み) &nbsp;:&nbsp;2018年09月
- 退会 &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;:&nbsp;2018年10月

# ライブラリ

In [47]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection
from dateutil.relativedelta import relativedelta

## データを読み込み

In [48]:
customer = pd.read_csv('customer_data.csv')       # 利用履歴を含んだ顧客データ
uselog_months = pd.read_csv('use_log_months.csv') # 利用履歴を年月、顧客毎に集計したデータ

## データ整形

In [49]:
# 当月、１ヶ月前の利用回数の集計データ
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


- 6ヶ月分のデータから予測した場合、ほんの数ヶ月辞めてしまう顧客を予測することができないので、当月と1ヶ月前のデータから予測します。

## 退会前月の退会顧客データを作成
退会の予測をする目的は、退会を未然に防ぐことです。  
退会申請を提出する1ヶ月前のデータから、翌月に退会申請を提出する確率を予測します。

In [50]:
# 退会を未然に防ぐ為に退会申請を出す、前月のデータを作成し"uselog"と結合
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# 欠損値除去
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


## 継続顧客のデータを作成

In [52]:
# 継続顧客データとuselogデータを結合し、欠損値除去
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


- nameカラムが欠損しているデータは、退会前月データと結合できていないため除去します。

In [53]:
# データをシャッフルし、"customer_id"が重複しているデータは最初のデータのみ取得
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201903,HI146430,5,5.0,XXXXX,C01,M,2015-06-01,NaN,CA1,...,オールタイム,10500.0,通常,4.583333,5.0,8.0,1.0,1.0,2019-04-30,46.0
1,201901,GD864253,9,6.0,XXXX,C02,F,2018-08-06,NaN,CA1,...,デイタイム,7500.0,通常,8.625000,8.5,13.0,6.0,1.0,2019-04-30,8.0
2,201808,AS019860,7,4.0,XXXXXX,C03,F,2017-10-01,NaN,CA1,...,ナイト,6000.0,通常,6.083333,6.0,9.0,4.0,1.0,2019-04-30,18.0
3,201903,GD629291,9,6.0,XXX,C01,M,2018-07-04,NaN,CA1,...,オールタイム,10500.0,通常,8.333333,8.0,10.0,6.0,1.0,2019-04-30,9.0
4,201902,AS672229,8,NaN,XXXX,C02,F,2019-02-08,NaN,CA1,...,デイタイム,7500.0,通常,8.500000,8.5,9.0,8.0,1.0,2019-04-30,2.0


- 退会データに対して継続データが多すぎるので、重複しているデータを1つに絞ります。

In [54]:
# 継続顧客データと退会顧客データを結合
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201903,HI146430,5,5.0,XXXXX,C01,M,2015-06-01,NaN,CA1,...,10500.0,通常,4.583333,5.0,8.0,1.0,1.0,2019-04-30,46.0,NaN
1,201901,GD864253,9,6.0,XXXX,C02,F,2018-08-06,NaN,CA1,...,7500.0,通常,8.625000,8.5,13.0,6.0,1.0,2019-04-30,8.0,NaN
2,201808,AS019860,7,4.0,XXXXXX,C03,F,2017-10-01,NaN,CA1,...,6000.0,通常,6.083333,6.0,9.0,4.0,1.0,2019-04-30,18.0,NaN
3,201903,GD629291,9,6.0,XXX,C01,M,2018-07-04,NaN,CA1,...,10500.0,通常,8.333333,8.0,10.0,6.0,1.0,2019-04-30,9.0,NaN
4,201902,AS672229,8,NaN,XXXX,C02,F,2019-02-08,NaN,CA1,...,7500.0,通常,8.500000,8.5,9.0,8.0,1.0,2019-04-30,2.0,NaN


## 予測する月の在籍期間を作成

In [55]:
# 在籍期間カラム作成
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201903,HI146430,5,5.0,XXXXX,C01,M,2015-06-01,NaN,CA1,...,4.583333,5.0,8.0,1.0,1.0,2019-04-30,46.0,NaN,45,2019-03-01
1,201901,GD864253,9,6.0,XXXX,C02,F,2018-08-06,NaN,CA1,...,8.625000,8.5,13.0,6.0,1.0,2019-04-30,8.0,NaN,4,2019-01-01
2,201808,AS019860,7,4.0,XXXXXX,C03,F,2017-10-01,NaN,CA1,...,6.083333,6.0,9.0,4.0,1.0,2019-04-30,18.0,NaN,10,2018-08-01
3,201903,GD629291,9,6.0,XXX,C01,M,2018-07-04,NaN,CA1,...,8.333333,8.0,10.0,6.0,1.0,2019-04-30,9.0,NaN,7,2019-03-01
4,201902,AS672229,8,NaN,XXXX,C02,F,2019-02-08,NaN,CA1,...,8.500000,8.5,9.0,8.0,1.0,2019-04-30,2.0,NaN,0,2019-02-01


## 欠損値を除去

In [56]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               275
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [57]:
# "count_1"が欠損しているデータのみ除去
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2619
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2619
period                  0
now_date                0
dtype: int64

## 文字列型の変数を処理できるように整形
文字列データを機械学習に対応させるために、数値化します。

In [58]:
# データ絞り込み
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,オールタイム,M,5.0,1.0,45,0.0
1,通常,デイタイム,F,6.0,1.0,4,0.0
2,通常,ナイト,F,4.0,1.0,10,0.0
3,通常,オールタイム,M,6.0,1.0,7,0.0
5,通常,デイタイム,F,5.0,1.0,15,0.0


In [59]:
# ダミー変数化
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,5.0,1.0,45,0.0,0,0,1,1,0,0,0,1
1,6.0,1.0,4,0.0,0,0,1,0,1,0,1,0
2,4.0,1.0,10,0.0,0,0,1,0,0,1,1,0
3,6.0,1.0,7,0.0,0,0,1,1,0,0,0,1
5,5.0,1.0,15,0.0,0,0,1,0,1,0,1,0


In [60]:
# ダミー変数化時にできた、使わない変数を除去
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,5.0,1.0,45,0.0,0,0,1,0,0
1,6.0,1.0,4,0.0,0,0,0,1,1
2,4.0,1.0,10,0.0,0,0,0,0,1
3,6.0,1.0,7,0.0,0,0,1,0,0
5,5.0,1.0,15,0.0,0,0,0,1,1


## 決定木を用いて退会予測モデルを作成

In [61]:
#継続と退会のデータ量を揃える(比率 : 50対50)
exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

# データ結合し、モデル作成
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1.
 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1.
 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1.
 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1.
 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0.
 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1.
 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0.
 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [62]:
# y_test, y_test_predを格納
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

,y_test,y_pred
1346,0.0,0.0
163,1.0,1.0
8,1.0,0.0
378,1.0,1.0
1509,0.0,0.0


- 先頭の5行は全て正解しています。

## 予測モデルの評価を行ない、モデルをチューニング

In [63]:
# results_testデータの正解率
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.9049429657794676


- 約90%正解しています。

In [64]:
# 学習用データと評価用データの精度
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9049429657794676
0.976552598225602


- 評価用データより、学習用データが約10%ほど高くなっているため過学習傾向にあると言えます。

In [65]:
# 前回のモデルを修正
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9125475285171103
0.9283903675538656


- 決定木の深さを5に設定し、モデルを簡易化することで精度の差を小さくしています。

## モデルに寄与している変数を確認

In [66]:
# 説明変数毎に係数を表示
importance = pd.DataFrame({"feature_names":X.columns, "coefficient":model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.338143
1,routine_flg,0.122123
2,period,0.538720
3,campaign_name_入会費半額,0.000000
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000000
6,class_name_デイタイム,0.000135
7,gender_F,0.000879


- １ヶ月前の利用回数が大きく寄与しており、その次に在籍期間、定期利用者かどうかが寄与していることが分かります。

## 顧客の退会を予測

In [67]:
# 変数を定義
count_1 = 3
routing_flg = 1
period = 10
campaign_name = "入会費無料"
class_name = "オールタイム"
gender = "M"

In [68]:
# ダミー変数を作成し、1つのリストに格納
if campaign_name == "入会費半額":
    campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
    campaign_name_list = [0, 1]
elif campaign_name == "通常":
    campaign_name_list = [0, 0]
if class_name == "オールタイム":
    class_name_list = [1, 0]
elif class_name == "デイタイム":
    class_name_list = [0, 1]
elif class_name == "ナイト":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routing_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [69]:
# 作成したデータを元に予測を行う
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

[1.]
[[0. 1.]]


- 今回の場合、退会(1)が予測されました。  
- 2行目で、継続(0)と退会(1)の予測確率が出力されています。